In [4]:
import pandas as pd
import re
from datetime import datetime

# Load the Excel file - assuming the data is in the first sheet
input_file = "agg.xlsx"  # Update with your actual file name
df = pd.read_excel(input_file)

# Function to extract data from complex regex_on_NER_result strings
def extract_bank_info(regex_result):
    if not isinstance(regex_result, str):
        return None, None, None, None, None, None
    
    bank_name = None
    account_no = None
    balance = None
    last_txn_date = None
    debit = None
    credit = None
    
    # Try to parse the JSON-like string
    try:
        # Extract bank name
        bank_match = re.search(r"'word':\s*'([^']*bank[^']*)'.*'entity':\s*'bank_accounts\.bank'", regex_result)
        if bank_match:
            bank_name = bank_match.group(1).strip()
        
        # Extract account number
        acct_match = re.search(r"'word':\s*'(\d+)'.*'entity':\s*'bank_accounts\.account'", regex_result)
        if acct_match:
            account_no = acct_match.group(1).strip()
        
        # Extract balance
        balance_match = re.search(r"'word':\s*'([^']*)'.*'entity':\s*'bank_accounts\.balance'", regex_result)
        if balance_match:
            balance = balance_match.group(1).replace(" ", "").replace(".", "")
        
        # Extract transaction date
        date_match = re.search(r"'word':\s*'([^']*)'.*'entity':\s*'bank_accounts\.last_txn_date'", regex_result)
        if date_match:
            last_txn_date = date_match.group(1).strip()
        
        # Extract debit amount
        debit_match = re.search(r"'word':\s*'([^']*)'.*'entity':\s*'bank_accounts\.debit'", regex_result)
        if debit_match:
            debit = debit_match.group(1).replace(" ", "").replace(".", "")
        
        # Extract credit amount
        credit_match = re.search(r"'word':\s*'([^']*)'.*'entity':\s*'bank_accounts\.credit'", regex_result)
        if credit_match:
            credit = credit_match.group(1).replace(" ", "").replace(".", "")
            
    except Exception as e:
        print(f"Error parsing: {e}")
        
    return bank_name, account_no, balance, last_txn_date, debit, credit

# Extract dictionary data format
def extract_dict_format(result_str):
    if not isinstance(result_str, str) or not result_str.startswith('{'):
        return None, None, None, None, None, None
    
    try:
        # Convert the string to a proper dictionary format
        # Replace single quotes with double quotes for JSON parsing
        result_str = result_str.replace("'", '"').replace("None", "null")
        
        # Get values without full JSON parsing (simpler regex approach)
        bank_name = re.search(r'"bank_accounts\.bank":\s*"([^"]*)"', result_str)
        bank_name = bank_name.group(1) if bank_name else None
        
        account_no = re.search(r'"bank_accounts\.account":\s*"([^"]*)"', result_str)
        account_no = account_no.group(1) if account_no else None
        
        balance = re.search(r'"bank_accounts\.balance":\s*"([^"]*)"', result_str)
        balance = balance.group(1) if balance else None
        
        last_txn_date = re.search(r'"bank_accounts\.last_txn_date":\s*"([^"]*)"', result_str)
        last_txn_date = last_txn_date.group(1) if last_txn_date else None
        
        debit = re.search(r'"bank_accounts\.debit":\s*"([^"]*)"', result_str)
        debit = debit.group(1) if debit else None
        
        credit = re.search(r'"bank_accounts\.credit":\s*"([^"]*)"', result_str)
        credit = credit.group(1) if credit else None
        
        return bank_name, account_no, balance, last_txn_date, debit, credit
    
    except Exception as e:
        print(f"Error parsing dictionary format: {e}")
        return None, None, None, None, None, None

# Create a new dataframe to hold the parsed data
parsed_data = []

# Process each row in the dataframe
for index, row in df.iterrows():
    # Check if we have a 'regex_on_NER_result' column
    if 'regex_on_NER_result' in df.columns:
        regex_result = row['regex_on_NER_result']
        bank_name, account_no, balance, txn_date, debit, credit = extract_bank_info(regex_result)
    elif 'Results' in df.columns:
        # Try both extraction methods
        bank_name, account_no, balance, txn_date, debit, credit = extract_bank_info(row['Results'])
        
        # If first method fails, try the dictionary format
        if bank_name is None and isinstance(row['Results'], str) and row['Results'].startswith('{'):
            bank_name, account_no, balance, txn_date, debit, credit = extract_dict_format(row['Results'])
    
    # If we have bank columns directly
    elif 'bank_name' in df.columns and 'account_no' in df.columns:
        bank_name = row['bank_name']
        account_no = row['account_no']
        balance = row.get('balance', None)
        txn_date = row.get('transaction_date', None)
        debit = row.get('debit_amount', None)
        credit = row.get('credit_amount', None)
    
    # Use existing values from specific columns if needed
    if 'bank_name' in df.columns and bank_name is None:
        bank_name = row['bank_name']
    
    if 'account_no' in df.columns and account_no is None:
        account_no = row['account_no'] 
        
    if 'balance' in df.columns and balance is None:
        balance = row['balance']
        
    if 'transaction_date' in df.columns and txn_date is None:
        txn_date = row['transaction_date']
        
    if 'debit_amount' in df.columns and debit is None:
        debit = row['debit_amount']
        
    if 'credit_amount' in df.columns and credit is None:
        credit = row['credit_amount']
    
    # Get the message date
    msg_date = row.get('msg_date', None)
    
    # Convert transaction date to standard format
    transaction_date = None
    if txn_date:
        # Try different date formats
        date_patterns = [
            r'(\d{2})\s*-\s*(\w{3})\s*-\s*(\d{2})',  # DD - MMM - YY
            r'(\d{2})(\w{3})',                       # DDMMM
        ]
        
        for pattern in date_patterns:
            match = re.search(pattern, str(txn_date))
            if match:
                if len(match.groups()) == 3:
                    day, month, year = match.groups()
                    month_dict = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                                'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
                    month_num = month_dict.get(month.lower(), 1)
                    transaction_date = f"20{year}-{month_num:02d}-{int(day):02d}"
                    break
                elif len(match.groups()) == 2:
                    day, month = match.groups()
                    month_dict = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                                'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
                    month_num = month_dict.get(month.lower(), 1)
                    transaction_date = f"2024-{month_num:02d}-{int(day):02d}"
                    break
    
    # If no transaction date found, use message date
    if transaction_date is None and msg_date and isinstance(msg_date, str):
        transaction_date = msg_date[:10]  # Just take the date part
    
    # Clean the data
    if account_no and isinstance(account_no, str):
        account_no = account_no.strip()
    
    if bank_name and isinstance(bank_name, str):
        bank_name = bank_name.strip().lower()
        # Standardize bank names
        if 'idbi' in bank_name:
            bank_name = 'idbi'
        elif 'robin' in bank_name or 'newton' in bank_name or 'rn' in bank_name:
            bank_name = 'robin newton bank'
    
    # Convert debit and credit to numeric values
    debit_amount = 0
    if debit and str(debit).strip():
        try:
            # Extract numeric values
            match = re.search(r'(\d+(?:\.\d+)?)', str(debit))
            if match:
                debit_amount = float(match.group(1))
        except:
            pass
    
    credit_amount = 0
    if credit and str(credit).strip():
        try:
            match = re.search(r'(\d+(?:\.\d+)?)', str(credit))
            if match:
                credit_amount = float(match.group(1))
        except:
            pass
    
    # Handle balance
    balance_amount = None
    if balance and str(balance).strip():
        try:
            match = re.search(r'(\d+(?:\.\d+)?)', str(balance))
            if match:
                balance_amount = float(match.group(1))
        except:
            pass
    
    # Add to parsed data if we have essential values
    if bank_name or account_no:
        parsed_data.append({
            'bank_name': bank_name,
            'account_no': account_no,
            'credit_amount': credit_amount,
            'debit_amount': debit_amount,
            'balance': balance_amount,
            'transaction_date': transaction_date
        })

# Create DataFrame from parsed data
bank_parsed = pd.DataFrame(parsed_data)

# Fill missing values
bank_parsed = bank_parsed.fillna({
    'credit_amount': 0,
    'debit_amount': 0
})

# Extract year-month for aggregation
bank_parsed['year_month'] = bank_parsed['transaction_date'].str[:7]  # Format: YYYY-MM

# Save the first file: bank_parsed.xlsx
bank_parsed.to_excel("bank_parsed.xlsx", index=False)

# Create aggregated data
# Group by account_no, bank_name, and year_month
aggregated = bank_parsed.groupby(['account_no', 'bank_name', 'year_month']).agg({
    'credit_amount': 'sum',
    'debit_amount': 'sum',
    'balance': 'last'  # Take the last balance value for the month
}).reset_index()

# Rename columns to match required format
bank_aggregated = aggregated.rename(columns={
    'credit_amount': 'total_credit',
    'debit_amount': 'total_debit'
})

# Save the second file: bank_aggregated.xlsx
bank_aggregated[['account_no', 'bank_name', 'total_credit', 'total_debit', 'year_month', 'balance']].to_excel(
    "bank_aggregated.xlsx", index=False
)

print("Data aggregation completed. Created bank_parsed.xlsx and bank_aggregated.xlsx")

Data aggregation completed. Created bank_parsed.xlsx and bank_aggregated.xlsx
